![youtube video](https://www.youtube.com/watch?v=B7oKdUvRhQQ&t=931s&ab_channel=FastbitEmbeddedBrainAcademy)
# What is a linker script?

- It is a text file that explains how different sections of the object files should be merged to create an output file
- Linker and locator 
- Linker script also includes code and data memory address and size information
- These scripts are written using _GNU linker command language_
- GNU linker script has the file extension of `.ld`
- You must supply linker script at the linking phase to the linker using `-T` option

# Basic linker script commands
 * Entry
 * Memory
 * Sections
 * Keep
 * Align
 * AT>

 ## Entry
 * This command is used to set the Entry point address information in th header of the final elf generated file
 * For a microcontroller, the `Reset_Handler` is the entry point into the application. This is the first piece of code that executes after a processor reset
 * The debugger uses this information to locate the first function to execute
 * This is not a mandatory command to use, but required when you debug the elf file using the debugger (GDB)
 * Syntax: Entry(_symbol_name_); Example: `Entry(Reset_Handler)`

 ## Memory 

 - Allows you to describe the different memories present in the target processor and their start address and size information
 - The linker uses information mentioned in this command to assign addresses to merged sections
 - Helps in data size verifications as this helps to calculate the total code and data memory so far consumed
 - Fine tune various memories available in the target and allow different sections to occupy different memory areas
 - Typiclaly one linker scipt has one memory command
### Syntax:
```
MEMORY
{
    name(attr) : ORIGIN = origin, LENGTH = len
}
```
label defines name of the memory region, will be ereferenced by other parts of the linker script
Origin: defines the start of the memory region
Length: Defines the length information

| Attribute Letter    | Meaning |
| -------- | ------- |
| R  | Read only sections    |
| W | Read and write sections     |
| X    | Sections containing executable code    |
| A    | Allocated sections    |
| I    | Initialized sections    |
| L    | Same as "I"    |
| !    | Invert the sense of any of the following attributes    |



![image](images/linkerMemoryMap1.png)

In the above code snippet we have taken the STM32F446RE microcontroller's memories and wrote them as three sections, namely, 
1. FLASH - with read (r) and executable (x) access only. We dont write to it, so we did not grant access
2. SRAM1 - with (rwx), meaning read, write and executable access
3. SRAM2 - with (rwx), meaning read, write and executable access

So, from the datasheet/technical refernce manual for the microcontroller, you should find
* How many memory spaces exist?
* Where are their starting origins?
* What is their length?

Also, the presenter decided to squish SRAM1 and SRAM2 into one memory space 

![image](images/linkerMemoryMap2.png)

So, we basically added `116 + 16 = 132 - 4 = 128`. So, why did we subtract 4KB? 

### Why did we subtract 4KB?
This is an important learning moment. Some microcontrollers have memory layout conventions that reserve some code memories or have other special functions. If we do not wish to use these memories, we exclude them from linker script. So, for the STM32 architecure, the manual says...

![image](images/embeddedSRAM_Note.png)

- This is why we subtracted 4KB as we are not writing to it, or interacting with it
- It’s not a linker script syntax requirement, but a platform-specific memory layout convention
- So, you need to read the microcontroller data sheet to assess whether you have such special features in the SRAM spaces 
- If you wish to intearct with this "back SRAM", you need to write it separately in the linker script

# Sections command
- `SECTIONS` command is used to create different output sections in the final elf executable generated
- This is an important commmand to instruct the linker script on how to merge the input sections to yield an output section
- This command also controls the order in which different output sections appear in the elf file generated
- You also mention the placement of a section in a memory region. For example, you instruct the linker to place the `.text` section in the `FLASH` memory region, which is described by the `MEMORY` command

![image](images/basic_SECTIONS_Command_Usage.png)

- In the above code, two sections namely `.text` and `.data` are declared
- This command is telling the linker to generate two sections `.text` and `.data` in the final `.elf` file
- You can give any name to labels like `.text`, but it is a good idea to follow standard naming conventions 
- But we know that according to gcc compilation, the memory sections names are standardized as `.data`, `.bss`, `.text`, `.rodata` etc...
- Next the end of each section `>(vma) AT>(lma)` gives the section placement information in the memory. They satdn for Virtual Memory Address and Load Memory Address

> Note that there are many files that have the `.text` section when they pass the compilation phase and manifest as `.o` files. So, the `.text` in linker takes all these individual sections and distills them into one contiiguous memory section. Same logic follows for all other sections.

Below image clarifies the above statement

![image](images/variousObjectsandTheirSections.png)


## Lets now start writing the sections commands

We will populate the data in the following order:
1. ISR vector section
2. Text section
3. Ro data
4. Data

Also note that we are writing all this data to the FLASH memory first. We must follow this order otherwise the application will not work properly. After each section of memory written, we stitch the sections using `> vma AT> lma`

So, lets define what vma and lma are first:
## VMA (Virtual Memory Address)
- Where the section/code/data "lives" when the program is running.

- It’s the address that the CPU uses to access the data/instruction during execution.

- It could be in RAM, ROM, Flash, or elsewhere depending on what the system sets up.

## LMA (Load Memory Address)
- Where the section/code/data is initially stored (often in non-volatile memory like Flash or ROM).

- It’s the physical location where the linker "places" the data so that the bootloader or startup code knows where to fetch it from.

- The loader (or your startup code) must copy data from LMA to VMA if they are different.


## Completing Sections command for .text:
Once you write the 
```
SECTIONS
{
    .text : 
    {
        *(.isr_section) // initial code space must contain the vector table. Note that we only have one ISR vector section
        *(.text) // add all the .text sections of the .text section from all object files
        *(.rodata) 
    }> FLASH AT> FLASH
}
```

we end it with the `vma` and `lma` specifications. When linker sees FLASH to FLASH specification, it generates the absolute addresses for the `.text` section. 

> If vma and lma are same you can just write `> memory name`. So something like `> FLASH`

## Next lets write the data section
This data section falls in FLASH, but the absolute address of the `.data` section will fall in the memory region SRAM 

```
    .data :
    {
        *(.data)
    }> SRAM AT> FLASH
```
## Next lets write the .bss data
This does not get stored in flash

```
    .bss
    {
        *(.bss)
    }> SRAM AT> FLASH

```

## Logical Reasoning Behind LMA/VMA Choices

### `.text` section
- **VMA**: `FLASH`
- **LMA**: `FLASH`
- **Reason**:
  - Program code (functions, ISRs) and constants are **executed directly from FLASH**.
  - STM32 microcontrollers support **Execute-In-Place (XIP)**, meaning code runs directly from FLASH without copying to RAM.
  - The **vector table** (`.isr_section`) must be at the beginning of FLASH so that the CPU can correctly fetch interrupt and reset vectors at startup.

### `.data` section
- **VMA**: `SRAM`
- **LMA**: `FLASH`
- **Reason**:
  - Initialized global and static variables (e.g., `int x = 5;`) must be **writable** during execution, so they must reside in **SRAM** at runtime.
  - However, their **initial values** need to be stored somewhere non-volatile; thus, they are initially placed in **FLASH**.
  - During startup, **boot code copies** the contents of the `.data` section from its LMA in FLASH to its VMA in SRAM.

### `.bss` section
- **VMA**: `SRAM`
- **LMA**: *(implicitly SRAM)*
- **Reason**:
  - Uninitialized global and static variables (e.g., `int y;`) are placed in the `.bss` section, which should be **zeroed** at runtime.
  - There's no need to store `.bss` in FLASH because it's just zeros.
  - During startup, **boot code zeros out** the `.bss` section in SRAM before `main()` starts.

---



## Location counter
We need to know the info on the boundaries of various data sections. These are generally for purposes like, 
* Where does my `.rodata` section start and end?
* What is the size of my `.data` section?

Below image shows the relocation of memory sections from FLASH to SRAM. If we dont know these boundaries, it is impossible to know what sections need to be copied where and which section we are copying at the current moment. 

![image](images/dataXfer_Sections.png)

### This is where the location counter comes to play...
- This is a special linker symbol denoted by a dot (`.`).
- This symbol is called the **"location counter"** since the linker automatically updates this symbol with location (address) information.
- You can use this symbol inside the linker script to **track and define boundaries** of various sections.
- You can also **set the location counter to any specific value** while writing the linker script.
- The location counter should **appear only inside the `SECTIONS` command**.
- The location counter is **incremented by the size of the output section**.

But before we can procees, we need to know about ** symbols **

### What is a symbol?
- A **symbol** is the **name of an address**.
- A **symbol declaration** is **not equivalent to a variable declaration** like in your `C` application.

![image](images/symbols.png)

Explanation:
Conside the main.c file and we have a variable declaration. We want to store value of 100 in the global variable. This anyways gets stored in data section. In the background, this will get converted to an address manipulation. So, how is this variable name replaced by address? That is what the symbol table does. When we compile the main.o, we compile the symbol table. For the compiler and liunler terminology, we call these symbol names. The symbol tabl;e has address and symbol name. So my value has a symbol name and an addres. So, the address is resolved by the symbol table. Again the fun1 definition belongs to the .text section. But this is again stored as a symbol in the symbol table. 

In C programs, we dont do symbol declarations. That's the compiler's job. But for the first time, we are actually going to manipulate and use the symbols in the linker script. These symbols pertain to the boundary informations. These are called linker script symbols. 

- A symbol is the name given for an address
- `__max_heap_size = 0x400;` When linker sees this entry, its going to add the entry to the symbol table. 
- we can also declare symbols inside the SECTIONS like catching the end of text section by writing `end_of_text = .;`. This is the location counter, which (remember) increments automatically. So, when you say `.`, the symbol will hold the end of the text section or whatever. 

![image](images/custom_symbols.png)

## If I am supposed to write the startup code, what is the point of a linker script anyways?
👉 Because the linker is the one who creates the binary.
It must:

- Know where each section will live (VMA = runtime address).

- Know where each section is stored (LMA = flash address).

- Emit special "symbols" (`_sdata`, `_edata`, `_sidata`, etc.) that your startup code can use to know:

- Where to copy from.

- Where to copy to.

- How much to copy.


What would happen if you didn't write it in the linker script?
- The linker would not reserve space properly for the sections.

- You would have no way to know (at runtime) where .data starts in Flash and RAM.

- Your startup code would be blind — no way to know source and destination addresses.

- Your binary image (Flash image) would be incorrectly laid out: .data contents might be missing or misplaced.

### In short:
The linker sets up the map. The startup code uses the map.

Without the linker script setting VMA and LMA correctly, your startup copy code would have garbage addresses and crash immediately.